In [1]:
import odrive
from odrive.enums import *
import time
import numpy as np
import import_ipynb

In [2]:
class od(object):
    '''
    Basic settings & commands ================================================================
    '''
    def connect_odrive(serial):
        odrv0 = odrive.find_any(serial_number = serial, timeout = 20)
        if odrv0 is not None:
            print('Connected!')
            print('Target serial {} \t Odrive serial {}'.format(serial, odrv0.serial_number))
            print('Motor 0 calibrated: {}'.format(odrv0.axis0.motor.is_calibrated))
            print('Motor 1 calibrated: {}'.format(odrv0.axis1.motor.is_calibrated))    
        else:
            print("ODrive ({}) not found".format(serial))
        return odrv0
    
    def SetParameter(odrv0):
        current_limit = 20
        odrv0.axis0.motor.config.current_lim    = current_limit #A
        odrv0.axis1.motor.config.current_lim    = current_limit #A
        odrv0.axis0.controller.config.vel_limit = 1000000
        odrv0.axis1.controller.config.vel_limit = 1000000
        odrv0.axis0.controller.config.pos_gain  = 60
        odrv0.axis1.controller.config.pos_gain  = 60
        odrv0.axis0.controller.config.vel_integrator_gain = 0.0005
        odrv0.axis1.controller.config.vel_integrator_gain = 0.0005
        return(current_limit)

    def CheckMotorError(odrv0):
        axis0error = odrv0.axis0.error 
        axis1error = odrv0.axis1.error 
        axis0motorerror = odrv0.axis0.motor.error 
        axis1motorerror = odrv0.axis1.motor.error 
        axis0controllererror = odrv0.axis0.controller.error 
        axis1controllererror = odrv0.axis0.controller.error 
        print("error:          ", axis0error, axis1error)
        print("motorerror:     ", axis0motorerror, axis1motorerror)
        print("controllererror:", axis0controllererror, axis1controllererror)
        return()

    def MotorClearError(odrv0):
        odrv0.axis0.error = 0
        odrv0.axis1.error = 0
        odrv0.axis0.motor.error = 0
        odrv0.axis1.motor.error = 0
        odrv0.axis0.controller.error = 0
        odrv0.axis1.controller.error = 0
        odrv0.axis0.encoder.error = 0
        odrv0.axis1.encoder.error = 0
        return()
    '''
    Motor commands ============================================================================
    '''
    def Calibration(odrv0):
        odrv0.axis0.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
        odrv0.axis1.requested_state = AXIS_STATE_FULL_CALIBRATION_SEQUENCE
        print('Motor 0 calibrated: {}'.format(odrv0.axis0.motor.is_calibrated))
        print('Motor 1 calibrated: {}'.format(odrv0.axis1.motor.is_calibrated))   

    def ClosedLoopControlMode(odrv0):
        odrv0.axis0.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL
        odrv0.axis1.requested_state = AXIS_STATE_CLOSED_LOOP_CONTROL

    def IdleMode(odrv0):
        odrv0.axis0.requested_state = odrive.enums.AXIS_STATE_IDLE
        odrv0.axis1.requested_state = odrive.enums.AXIS_STATE_IDLE

    def ControllerPositionControl(odrv0): # default mode
        odrv0.axis0.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
        odrv0.axis1.controller.config.control_mode = CTRL_MODE_POSITION_CONTROL
    #     print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

    def ControllerCurrentControl(odrv0):
        odrv0.axis0.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
        odrv0.axis1.controller.config.control_mode = CTRL_MODE_CURRENT_CONTROL
    #     print("Control Mode :{}".format(odrv0.axis0.controller.config.control_mode))

    def MaxCurrentProtection(current, current_limit = 20):
        current = min(np.abs(current), current_limit) * np.sign(current) 
        return(current)
    '''
    Defining angles, run before test ==========================================================
    '''
    def SetMotorHomePosition(m0, m1):
        m0_Home_pos = m0.encoder.pos_estimate
        m1_Home_pos = m1.encoder.pos_estimate
        theta_R_Home = -((m0_Home_pos) / 8192 * 2 * np.pi) + np.pi
        theta_L_Home = -((m1_Home_pos) / 8192 * 2 * np.pi)
        return m0_Home_pos, m1_Home_pos 

    def SetMotorEquilibriumPosition(m0, m1, m0_Home_pos, m1_Home_pos):
        m0_eq_toe_pos = m0.encoder.pos_estimate
        m1_eq_toe_pos = m1.encoder.pos_estimate
        theta_R_eq = - ((m0_eq_toe_pos - m0_Home_pos) / 8192 * 2 * np.pi) + np.pi
        theta_L_eq = - ((m1_eq_toe_pos - m1_Home_pos) / 8192 * 2 * np.pi)
        theta_R_eq_d = theta_R_eq / (2 * np.pi) * 360
        theta_L_eq_d = theta_L_eq / (2 * np.pi) * 360
        return theta_R_eq_d, theta_L_eq_d

In [3]:
import math
import sympy as sp
from sympy.utilities.lambdify import lambdify
import matplotlib.pyplot as plt
from kinematics_function import RobotKinematics, InvereseKinematics
from kinematics_function import move

importing Jupyter notebook from kinematics_function.ipynb


In [4]:
(thetaL_sym, thetaR_sym) = sp.symbols("""thetaL_sym thetaR_sym """, real = True)

FK = RobotKinematics()[0]
FK_fast = lambdify((thetaR_sym, thetaL_sym), FK) # (x,y) = FK_fast(thetaR,thetaL)
J = RobotKinematics()[1]
J_fast = lambdify((thetaR_sym, thetaL_sym), J) # J_fast(thetaR,thetaL)

In [5]:
class gait(object):
    '''
    Space convertor =========================================================================
    '''
    #constructing--------------------!!
    def from_joint_get_pos(m0_home_pos, m1_home_pos): #return current xy

        thetaR = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
        thetaL = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)

        x = FK_fast(thetaR, thetaL)[0]
        y = FK_fast(thetaR, thetaL)[1]

        return np.array([x, y])

    def from_pos_get_joint(x, y, m0_home_pos, m1_home_pos): #xy -> encoder

        theta_R = InvereseKinematics(x, y)[1]
        theta_L = InvereseKinematics(x, y)[0]

        m0_pos = -(theta_R - np.pi)/(2*np.pi)*8192 + m0_home_pos
        m1_pos = -theta_L/(2*np.pi)*8192 + m1_home_pos

        return np.array([m0_pos, m1_pos])
    
    def get_ready_pos(UA, DA, SL, odrv1, odrv2, m0, m1, m00, m11, m0_home_pos, m1_home_pos, m00_home_pos, m11_home_pos):
        
        od.ControllerPositionControl(odrv1)
        od.ControllerPositionControl(odrv2)

        od.ClosedLoopControlMode(odrv1)
        od.ClosedLoopControlMode(odrv2)

        # Get ready to start---------------------------------------------------
        phase_t = 1

        m0_pos = gait.from_pos_get_joint(move.RightStanceTrajectory(0, phase_t, DA, SL)[0], move.RightStanceTrajectory(0, phase_t, DA, SL)[1]-0.02, m0_home_pos, m1_home_pos)[0]
        m1_pos = gait.from_pos_get_joint(move.RightStanceTrajectory(0, phase_t, DA, SL)[0], move.RightStanceTrajectory(0, phase_t, DA, SL)[1]-0.02, m0_home_pos, m1_home_pos)[1]
        m00_pos= gait.from_pos_get_joint(move.LeftSwingTrajectory(0, phase_t, UA, SL)[0], move.LeftSwingTrajectory(0, phase_t, UA, SL)[1]-0.01, m00_home_pos, m11_home_pos)[0]
        m11_pos= gait.from_pos_get_joint(move.LeftSwingTrajectory(0, phase_t, UA, SL)[0], move.LeftSwingTrajectory(0, phase_t, UA, SL)[1]-0.01, m00_home_pos, m11_home_pos)[1]

        m0.controller.set_pos_setpoint(m0_pos,0,0)
        m1.controller.set_pos_setpoint(m1_pos,0,0)
        m00.controller.set_pos_setpoint(m00_pos,0,0)
        m11.controller.set_pos_setpoint(m11_pos,0,0)
    '''
    Test gait ================================================================================
    '''
    def run_positional_step_test(test_cycle, swing_time, stance_time, UA, DA, SL, odrv1, odrv2, m0, m1, m00, m11, m0_home_pos, m1_home_pos, m00_home_pos, m11_home_pos, pause_time = 0):

        #1 IS RIGHT LEG, 2 IS LEFT LEG
        x1 = []
        y1 = []
        x2 = []
        y2 = []
        x1_real = []
        y1_real = []
        x2_real = []
        y2_real = []

        od.ClosedLoopControlMode(odrv1)
        od.ClosedLoopControlMode(odrv2)
        count = 1
        period = swing_time

        while count <= test_cycle:
            '''
            Pausing =================================================================================
            '''
            t_pause_start = time.time()
            while time.time() - t_pause_start < pause_time:

                pass
            '''
            RightStance LeftSwing Trajactory ========================================================================
            '''
            t_swing_start = time.time()
            swing_count = 0
            while time.time() - t_swing_start < swing_time:

                t_clock = time.time() - t_swing_start

                theta_R1 = InvereseKinematics(move.RightStanceTrajectory(t_clock, period, DA, SL)[0],move.RightStanceTrajectory(t_clock, period, DA, SL)[1])[1]
                theta_L1 = InvereseKinematics(move.RightStanceTrajectory(t_clock, period, DA, SL)[0],move.RightStanceTrajectory(t_clock, period, DA, SL)[1])[0]
                theta_R2 = InvereseKinematics(move.LeftSwingTrajectory(t_clock, period, UA, SL)[0],move.LeftSwingTrajectory(t_clock, period, UA, SL)[1])[1]
                theta_L2 = InvereseKinematics(move.LeftSwingTrajectory(t_clock, period, UA, SL)[0],move.LeftSwingTrajectory(t_clock, period, UA, SL)[1])[0]

                m0_pos  = -(theta_R1 - np.pi)/(2*np.pi)*8192 + m0_home_pos
                m1_pos  = -theta_L1/(2*np.pi)*8192 + m1_home_pos
                m00_pos = -(theta_R2 - np.pi)/(2*np.pi)*8192 + m00_home_pos
                m11_pos = -theta_L2/(2*np.pi)*8192 + m11_home_pos

                m0.controller.set_pos_setpoint(m0_pos,0,0)
                m1.controller.set_pos_setpoint(m1_pos,0,0)
                m00.controller.set_pos_setpoint(m00_pos,0,0)
                m11.controller.set_pos_setpoint(m11_pos,0,0)

                #recording (1 IS RIGHT LEG, 2 IS LEFT LEG)--------------------------------------------------
                x1.append(move.RightStanceTrajectory(t_clock, period, DA, SL)[0])
                y1.append(move.RightStanceTrajectory(t_clock, period, DA, SL)[1])
                x2.append(move.LeftSwingTrajectory(t_clock, period, UA, SL)[0])
                y2.append(move.LeftSwingTrajectory(t_clock, period, UA, SL)[1])

                real_theta_R1 = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L1 = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
                real_theta_R2 = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L2 = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
                x1_real.append(FK_fast(real_theta_R1, real_theta_L1)[0])
                y1_real.append(FK_fast(real_theta_R1, real_theta_L1)[1])
                x2_real.append(FK_fast(real_theta_R2, real_theta_L2)[0])
                y2_real.append(FK_fast(real_theta_R2, real_theta_L2)[1])

                swing_count += 1

            '''
            LeftStance RightSwing Trajactory ========================================================================
            '''
            t_stance_start = time.time()
            stance_count = 0
            while time.time() - t_stance_start < stance_time:

                # 1 IS RIGHT LEG, 2 IS LEFT LEG
                t_clock = time.time() - t_stance_start

                theta_R1 = InvereseKinematics(move.RightSwingTrajectory(t_clock, period, UA, SL)[0],move.RightSwingTrajectory(t_clock, period, UA, SL)[1])[1]
                theta_L1 = InvereseKinematics(move.RightSwingTrajectory(t_clock, period, UA, SL)[0],move.RightSwingTrajectory(t_clock, period, UA, SL)[1])[0]
                theta_R2 = InvereseKinematics(move.LeftStanceTrajectory(t_clock, period, DA, SL)[0],move.LeftStanceTrajectory(t_clock, period, DA, SL)[1])[1]
                theta_L2 = InvereseKinematics(move.LeftStanceTrajectory(t_clock, period, DA, SL)[0],move.LeftStanceTrajectory(t_clock, period, DA, SL)[1])[0]

                m0_pos  = -(theta_R1 - np.pi)/(2*np.pi)*8192 + m0_home_pos
                m1_pos  = -theta_L1/(2*np.pi)*8192 + m1_home_pos
                m00_pos = -(theta_R2 - np.pi)/(2*np.pi)*8192 + m00_home_pos
                m11_pos = -theta_L2/(2*np.pi)*8192 + m11_home_pos

                m0.controller.set_pos_setpoint(m0_pos,0,0)
                m1.controller.set_pos_setpoint(m1_pos,0,0)
                m00.controller.set_pos_setpoint(m00_pos,0,0)
                m11.controller.set_pos_setpoint(m11_pos,0,0)

                #recording (1 IS RIGHT LEG, 2 IS LEFT LEG)--------------------------------------------------
                x1.append(move.RightSwingTrajectory(t_clock, period, UA, SL)[0])
                y1.append(move.RightSwingTrajectory(t_clock, period, UA, SL)[1])
                x2.append(move.LeftStanceTrajectory(t_clock, period, DA, SL)[0])
                y2.append(move.LeftStanceTrajectory(t_clock, period, DA, SL)[1])

                real_theta_R1 = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L1 = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
                real_theta_R2 = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L2 = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
                x1_real.append(FK_fast(real_theta_R1, real_theta_L1)[0])
                y1_real.append(FK_fast(real_theta_R1, real_theta_L1)[1])
                x2_real.append(FK_fast(real_theta_R2, real_theta_L2)[0])
                y2_real.append(FK_fast(real_theta_R2, real_theta_L2)[1])

                stance_count += 1

            count += 1

        od.ControllerPositionControl(odrv1)
        od.ControllerPositionControl(odrv2)

        od.IdleMode(odrv1)
        od.IdleMode(odrv2)

        # Plotting-----------------------------------------------------------------------------------
        plt.subplot(1,2,1)
        plt.title("Right Leg")
        plt.plot(x1,y1)
        plt.plot(x1_real,y1_real)
        plt.legend(['commanded point','real point'])   
        plt.xlim(0.1, -0.1)
        plt.ylim(0.2, 0.09)
        plt.text(0.16, 0.14,'y(m)',va='center',rotation='vertical')
        plt.grid()

        plt.subplot(1,2,2)
        plt.title("Left Leg")
        plt.plot(x2,y2)
        plt.plot(x2_real,y2_real)
        plt.legend(['commanded point','real point'])
        plt.xlim(0.1, -0.1)
        plt.ylim(0.2, 0.09)
        plt.text(0.12, 0.21,'x(m)',ha='center')
        plt.text(0.12, 0.08,'Positional Control',ha='center')
        plt.grid()

        plt.show()
    '''
    =============================================================================================================
    '''
    
    def run_aniso_compliance_step_test(test_cycle, kval, swing_time, stance_time, sr, cgr, kill_motor, UA, DA, SL, odrv1, odrv2, m0, m1, m00, m11, m0_home_pos, m1_home_pos, m00_home_pos, m11_home_pos, pause_time = 0):

        #1 IS RIGHT LEG, 2 IS LEFT LEG
        x1 = []
        y1 = []
        x2 = []
        y2 = []
        x1_real = []
        y1_real = []
        x2_real = []
        y2_real = []        
        m0_StanceTorque = []
        m1_StanceTorque = []
        m00_StanceTorque = []
        m11_StanceTorque = []

        od.ClosedLoopControlMode(odrv1)
        od.ClosedLoopControlMode(odrv2)
        count = 1
        period = swing_time

        while count <= test_cycle:
            '''
            Pausing =================================================================================
            '''
            t_pause_start = time.time()
            while time.time() - t_pause_start < pause_time:

                pass
            '''
            RightStance LeftSwing Trajactory ========================================================================
            '''
            t_swing_start = time.time()
            swing_count = 0
            while time.time() - t_swing_start < swing_time:

                t_clock = time.time() - t_swing_start

                theta_R1 = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
                theta_R2 = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
                theta_L1 = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
                theta_L2 = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)

                vel_R1 = - (m0.encoder.vel_estimate / 8192 * 2 * np.pi)
                vel_R2 = - (m00.encoder.vel_estimate / 8192 * 2 * np.pi)
                vel_L1= - (m1.encoder.vel_estimate / 8192 * 2 * np.pi)
                vel_L2 = - (m11.encoder.vel_estimate / 8192 * 2 * np.pi)

                current1 = move.anisotropic_antigravity_rightstance(kval,theta_R1, theta_L1, vel_R1, vel_L1, t_swing_start, period, DA, SL, cgr)
                current2 = move.anisotropic_antigravity_leftswing(kval,theta_R2, theta_L2, vel_R2, vel_L2, t_swing_start, period, sr, UA, SL, cgr)

                # Motor current = Current - Damp Current----------------------------------------------------
                m0_current_R = current1[0] - np.array(0.5 * vel_R1)
                m1_current_L = current1[1] - np.array(0.5 * vel_L1)
                m00_current_R = current2[0] - np.array(0.5 * vel_R2)
                m11_current_L = current2[1] - np.array(0.5 * vel_L2)

                m0.controller.set_current_setpoint(-od.MaxCurrentProtection(m0_current_R))
                m1.controller.set_current_setpoint(-od.MaxCurrentProtection(m1_current_L)*kill_motor)
                m00.controller.set_current_setpoint(-od.MaxCurrentProtection(m00_current_R)*kill_motor)
                m11.controller.set_current_setpoint(-od.MaxCurrentProtection(m11_current_L))

                #recording (1 IS RIGHT LEG, 2 IS LEFT LEG)--------------------------------------------------
                x1.append(move.RightStanceTrajectory(t_clock, period, DA, SL)[0])
                y1.append(move.RightStanceTrajectory(t_clock, period, DA, SL)[1])
                x2.append(move.LeftSwingTrajectory(t_clock, period, UA, SL)[0])
                y2.append(move.LeftSwingTrajectory(t_clock, period, UA, SL)[1])

                real_theta_R1 = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L1 = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
                real_theta_R2 = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L2 = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
                x1_real.append(FK_fast(real_theta_R1, real_theta_L1)[0])
                y1_real.append(FK_fast(real_theta_R1, real_theta_L1)[1])
                x2_real.append(FK_fast(real_theta_R2, real_theta_L2)[0])
                y2_real.append(FK_fast(real_theta_R2, real_theta_L2)[1])
                
                m0_StanceTorque.append(8.27*m0.motor.current_control.Iq_measured/335)
                m1_StanceTorque.append(8.27*m1.motor.current_control.Iq_measured/335)
                
                swing_count += 1

            '''
            LeftStance RightSwing Trajactory ========================================================================
            '''
            t_stance_start = time.time()
            stance_count = 0
            while time.time() - t_stance_start < stance_time:

                # 1 IS RIGHT LEG, 2 IS LEFT LEG
                t_clock = time.time() - t_stance_start
                theta_R2 = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
                theta_R1 = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
                theta_L2 = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
                theta_L1 = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)

                vel_R1 = - (m0.encoder.vel_estimate / 8192 * 2 * np.pi)
                vel_R2 = - (m00.encoder.vel_estimate / 8192 * 2 * np.pi)
                vel_L1 = - (m1.encoder.vel_estimate / 8192 * 2 * np.pi)
                vel_L2 = - (m11.encoder.vel_estimate / 8192 * 2 * np.pi)

                current2 = move.anisotropic_antigravity_leftstance(kval,theta_R2, theta_L2, vel_R2, vel_L2, t_stance_start, period, DA, SL, cgr)
                current1 = move.anisotropic_antigravity_rightswing(kval,theta_R1, theta_L1, vel_R1, vel_L1, t_stance_start, period, sr, UA, SL, cgr)

                # Motor current = Current - Damp Current---------------------------------------------------
                m0_current_R = current1[0] - np.array(0.5 * vel_R1)
                m1_current_L = current1[1] - np.array(0.5 * vel_L1)
                m00_current_R = current2[0] - np.array(0.5 * vel_R2)
                m11_current_L = current2[1] - np.array(0.5 * vel_L2)

                m0.controller.set_current_setpoint(-od.MaxCurrentProtection(m0_current_R))
                m1.controller.set_current_setpoint(-od.MaxCurrentProtection(m1_current_L))
                m00.controller.set_current_setpoint(-od.MaxCurrentProtection(m00_current_R))
                m11.controller.set_current_setpoint(-od.MaxCurrentProtection(m11_current_L))

                #recording (1 IS RIGHT LEG, 2 IS LEFT LEG)--------------------------------------------------
                x1.append(move.RightSwingTrajectory(t_clock, period, UA, SL)[0])
                y1.append(move.RightSwingTrajectory(t_clock, period, UA, SL)[1])
                x2.append(move.LeftStanceTrajectory(t_clock, period, DA, SL)[0])
                y2.append(move.LeftStanceTrajectory(t_clock, period, DA, SL)[1])

                real_theta_R1 = - ((m0.encoder.pos_estimate - m0_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L1 = - ((m1.encoder.pos_estimate - m1_home_pos) / 8192 * 2 * np.pi)
                real_theta_R2 = - ((m00.encoder.pos_estimate - m00_home_pos) / 8192 * 2 * np.pi) + np.pi
                real_theta_L2 = - ((m11.encoder.pos_estimate - m11_home_pos) / 8192 * 2 * np.pi)
                x1_real.append(FK_fast(real_theta_R1, real_theta_L1)[0])
                y1_real.append(FK_fast(real_theta_R1, real_theta_L1)[1])
                x2_real.append(FK_fast(real_theta_R2, real_theta_L2)[0])
                y2_real.append(FK_fast(real_theta_R2, real_theta_L2)[1])
                
                m00_StanceTorque.append(8.27*m00.motor.current_control.Iq_measured/335)
                m11_StanceTorque.append(8.27*m11.motor.current_control.Iq_measured/335)

                stance_count += 1

            count += 1

        m00.controller.set_current_setpoint(0)
        m11.controller.set_current_setpoint(0)
        m0.controller.set_current_setpoint(0)
        m1.controller.set_current_setpoint(0)

        od.ControllerPositionControl(odrv1)
        od.ControllerPositionControl(odrv2)

        od.IdleMode(odrv1)
        od.IdleMode(odrv2)
        
        # Torque data post-process (Torque [N.m] = 8.27 * Current [A] / KV.)-------------------------
        odrv1_torque_data = np.zeros((len(m0_StanceTorque),2))
        odrv2_torque_data = np.zeros((len(m00_StanceTorque),2))
        
        odrv1_torque_data[:,0] = m0_StanceTorque
        odrv1_torque_data[:,1] = m1_StanceTorque
        odrv2_torque_data[:,0] = m00_StanceTorque
        odrv2_torque_data[:,1] = m11_StanceTorque

        # Plotting-----------------------------------------------------------------------------------
        plt.subplot(2,2,1)
        plt.title("Right Leg")
        plt.plot(x1,y1)
        plt.plot(x1_real,y1_real)
        plt.legend(['commanded point','real point'])   
        plt.xlim(0.1, -0.1)
        plt.ylim(0.2, 0.09)
        plt.text(0.16, 0.14,'y(m)',va='center',rotation='vertical')
        plt.grid()

        plt.subplot(2,2,2)
        plt.title("Left Leg")
        plt.plot(x2,y2)
        plt.plot(x2_real,y2_real)
        plt.legend(['commanded point','real point'])
        plt.xlim(0.1, -0.1)
        plt.ylim(0.2, 0.09)
        plt.text(0.12, 0.21,'x(m)',ha='center')
        plt.text(0.12, 0.08,'Anisotropic k ratio={}, k={}, cycle={}'.format(sr,kval,test_cycle),ha='center')
        plt.grid()
        
        plt.subplot(2,2,3)
        plt.title("Right Leg Odrv1 Stance Torque (N*m)")
        plt.plot(np.arange(len(m0_StanceTorque)),odrv1_torque_data[:,0])
        plt.plot(np.arange(len(m1_StanceTorque)),odrv1_torque_data[:,1])
        plt.legend(['m0 (front)','m1'])
        plt.grid()
        
        plt.subplot(2,2,4)
        plt.title("Left Leg Odrv2 Stance Torque (N*m)")
        plt.plot(np.arange(len(m00_StanceTorque)),odrv2_torque_data[:,0])
        plt.plot(np.arange(len(m11_StanceTorque)),odrv2_torque_data[:,1])
        plt.legend(['m00','m11 (front)'])
        plt.grid()
        
        plt.show()